# FailureSense – Artificial Neural Network (ANN)

Objective:
- Implement a basic ANN for failure prediction
- Compare ANN performance with classical ML models
- Evaluate ANN under class imbalance


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [4]:
df = pd.read_csv(r"D:\FailureSense_MLProj\failuresense\data\raw\ai4i2020.csv")

In [5]:
DROP_COLUMNS = ["UDI", "TWF", "HDF", "PWF", "OSF", "RNF"]
TARGET = "Machine failure"

X = df.drop(columns=DROP_COLUMNS + [TARGET])
y = df[TARGET]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


Preprocessing for ANN

Encode + scale explicitly:

In [8]:
numeric_features = [
    "Air temperature [K]",
    "Process temperature [K]",
    "Rotational speed [rpm]",
    "Torque [Nm]",
    "Tool wear [min]"
]

categorical_features = ["Type"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_features)
    ]
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


Build ANN model

In [9]:
model = Sequential([
    Dense(16, activation="relu", input_shape=(X_train_processed.shape[1],)),
    Dense(8, activation="relu"),
    Dense(1, activation="sigmoid")
])


C:\Users\ACSS\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


compile model

In [11]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


train ann

In [12]:
history = model.fit(
    X_train_processed,
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9314 - loss: 0.3342 - val_accuracy: 0.9656 - val_loss: 0.1731
Epoch 2/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9663 - loss: 0.1533 - val_accuracy: 0.9656 - val_loss: 0.1362
Epoch 3/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9670 - loss: 0.1271 - val_accuracy: 0.9663 - val_loss: 0.1193
Epoch 4/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9689 - loss: 0.1110 - val_accuracy: 0.9669 - val_loss: 0.1080
Epoch 5/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9692 - loss: 0.1005 - val_accuracy: 0.9688 - val_loss: 0.1017
Epoch 6/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9700 - loss: 0.0941 - val_accuracy: 0.9681 - val_loss: 0.0987
Epoch 7/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9716 - loss: 0.0903 - val_accuracy: 0.9706 - val_loss: 0.0961
Epoch 8/20
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9734 - loss: 0.0878 - val_accuracy: 0.

evaluate ann

In [13]:
y_prob_ann = model.predict(X_test_processed).ravel()
y_pred_ann = (y_prob_ann >= 0.5).astype(int)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [14]:
confusion_matrix(y_test, y_pred_ann)


array([[1921,   11],
       [  43,   25]])

In [15]:
print(classification_report(y_test, y_pred_ann))


              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1932
           1       0.69      0.37      0.48        68

    accuracy                           0.97      2000
   macro avg       0.84      0.68      0.73      2000
weighted avg       0.97      0.97      0.97      2000



In [16]:
roc_auc_score(y_test, y_prob_ann)


0.9510488978199976

### ANN Observations

- ANN captures non-linear relationships
- Performance comparable to ensemble methods on tabular data
- Requires careful preprocessing and tuning
- Increased complexity without guaranteed superiority
